# Sports Tournament Scheduler - Genetic Algorithm

This notebook implements a complete Sports Tournament Scheduling system using a Genetic Algorithm.

## Project Structure
- All code is contained in this single notebook
- Code runs sequentially from top to bottom
- GUI can be launched directly from a notebook cell


## Cell 1: Imports and Global Settings


In [1]:
import random
import copy
from datetime import datetime, timedelta
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import threading
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Set matplotlib backend for notebook
%matplotlib inline


## Cell 2: Data Definitions
 

In [2]:
# Teams list
TEAMS = [
    "Al Ahly", "Zamalek", "Pyramids", "Masry", "Future", "Ismaily",
    "Smouha", "ENPPI", "Ceramica", "National Bank", "Talaea El Gaish",
    "Alexandria Union", "El Dakhleya", "El Gouna", "Zed",
    "Modern Sport", "Pharco", "Wadi Degla"
]

# Venues list
VENUES = [
    "Cairo Stadium", "Borg El Arab", "Air Defense Stadium",
    "Suez Stadium", "Alexandria Stadium", "Petro Sport Stadium",
    "Military Academy Stadium", "Al Salam Stadium",
    "El Sekka El Hadeed Stadium", "Zed Club Stadium"
]

# Match times
MATCH_TIMES = ["17:00", "20:00"]

# Generate dates
def generate_dates(start_date=None, end_date=None):
    """Generate list of available dates for the tournament"""
    if start_date is None:
        start_date = datetime(2025, 5, 1)
    if end_date is None:
        end_date = datetime(2026, 1, 31)
    
    dates = []
    current = start_date
    while current <= end_date:
        dates.append(current)
        current += timedelta(days=1)
    return dates

# Generate dates
DATES = generate_dates()


## Cell 3: Match Class


In [3]:
class Match:
    """Represents a single match in the tournament schedule"""
    
    def __init__(self, team1, team2, date, time, venue, leg):
        self.team1 = team1
        self.team2 = team2
        self.date = date
        self.time = time
        self.venue = venue
        self.leg = leg

    def __repr__(self):
        return f"{self.team1} vs {self.team2} (leg {self.leg}) on {self.date.strftime('%Y-%m-%d')} {self.time} at {self.venue}"
    
    def __eq__(self, other):
        if not isinstance(other, Match):
            return False
        return (self.team1 == other.team1 and 
                self.team2 == other.team2 and 
                self.leg == other.leg)
    
    def __hash__(self):
        return hash((self.team1, self.team2, self.leg))


## Cell 4: Encoding & Population Initialization


In [4]:
def generate_all_matches(teams):
    """Generate all possible matches (home & away) for the tournament"""
    matches = []
    for i in range(len(teams)):
        for j in range(i + 1, len(teams)):
            matches.append((teams[i], teams[j], 1))  # Leg 1
            matches.append((teams[j], teams[i], 2))  # Leg 2
    return matches


def create_random_individual(teams, venues, dates, match_times):
    """Create a random chromosome (individual) representing a full schedule"""
    chromosome = []
    all_matches = generate_all_matches(teams)
    random.shuffle(all_matches)

    for team1, team2, leg in all_matches:
        chromosome.append(
            Match(
                team1,
                team2,
                random.choice(dates),
                random.choice(match_times),
                random.choice(venues),
                leg
            )
        )
    return chromosome


def create_initial_population(pop_size, teams, venues, dates, match_times):
    """Create initial population of random individuals"""
    population = []
    for _ in range(pop_size):
        population.append(
            create_random_individual(teams, venues, dates, match_times)
        )
    return population


## Cell 5: Fitness Function


In [5]:
def fitness(individual):
    """
    Calculate fitness of an individual (schedule).
    
    Fitness = 1 / (1 + total_penalty)
    
    Penalties:
    - Venue Conflicts (weight 5): Same venue + same date + same time
    - Rest Period Violations (weight 3): Team playing with less than 1 day rest
    - Match Time Balance (weight 1): Uneven distribution between 17:00 and 20:00
    """
    penalty = 0

    # 1. Venue Conflicts (weight 5)
    venue_time_dict = {}
    for match in individual:
        key = (match.date, match.time, match.venue)
        if key in venue_time_dict:
            penalty += 5
        else:
            venue_time_dict[key] = match

    # 2. Rest periods (weight 3)
    team_dates = {}
    for match in individual:
        for team in [match.team1, match.team2]:
            if team not in team_dates:
                team_dates[team] = []
            team_dates[team].append(match.date)

    for dates in team_dates.values():
        dates.sort()
        for i in range(1, len(dates)):
            if (dates[i] - dates[i-1]).days < 1:
                penalty += 3

    # 3. Balance game times (weight 1)
    team_time_count = {team: {"17:00": 0, "20:00": 0} for team in team_dates}
    for match in individual:
        team_time_count[match.team1][match.time] += 1
        team_time_count[match.team2][match.time] += 1

    for counts in team_time_count.values():
        penalty += abs(counts["17:00"] - counts["20:00"]) * 1

    # Final fitness (higher is better)
    return 1 / (1 + penalty)


def calculate_penalty_breakdown(individual):
    """
    Calculate detailed penalty breakdown for analysis.
    Returns dict with individual penalty components.
    """
    venue_conflicts = 0
    rest_violations = 0
    time_imbalance = 0
    
    # 1. Venue Conflicts
    venue_time_dict = {}
    for match in individual:
        key = (match.date, match.time, match.venue)
        if key in venue_time_dict:
            venue_conflicts += 1
        else:
            venue_time_dict[key] = match

    # 2. Rest periods
    team_dates = defaultdict(list)
    for match in individual:
        team_dates[match.team1].append(match.date)
        team_dates[match.team2].append(match.date)

    for dates_list in team_dates.values():
        dates_list.sort()
        for i in range(1, len(dates_list)):
            if (dates_list[i] - dates_list[i-1]).days < 1:
                rest_violations += 1

    # 3. Time balance
    team_time_count = {team: {"17:00": 0, "20:00": 0} for team in team_dates}
    for match in individual:
        team_time_count[match.team1][match.time] += 1
        team_time_count[match.team2][match.time] += 1

    for counts in team_time_count.values():
        time_imbalance += abs(counts["17:00"] - counts["20:00"])
    
    total_penalty = venue_conflicts * 5 + rest_violations * 3 + time_imbalance
    
    return {
        'venue_conflicts': venue_conflicts,
        'rest_violations': rest_violations,
        'time_imbalance': time_imbalance,
        'total_penalty': total_penalty,
        'fitness': 1 / (1 + total_penalty)
    }


## Cell 6: Selection Methods


In [6]:
def tournament_selection(population, fitness_scores, k=3):
    """
    Tournament selection: Select k individuals randomly and return the best.
    Balances exploration and exploitation.
    """
    selected = random.sample(list(zip(population, fitness_scores)), k)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]


def roulette_wheel_selection(population, fitness_scores):
    """
    Roulette wheel selection: Select individual proportionally to fitness.
    Higher fitness = higher probability of selection.
    """
    total_fitness = sum(fitness_scores)
    pick = random.uniform(0, total_fitness)
    current = 0
    for ind, fit in zip(population, fitness_scores):
        current += fit
        if current > pick:
            return ind
    return population[-1]  # Fallback


## Cell 7: Crossover Methods


In [7]:
def one_point_crossover(parent1, parent2):
    """
    One-point crossover: Exchange segments at a random point.
    """
    point = random.randint(1, len(parent1) - 2)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2


def two_point_crossover(parent1, parent2):
    """
    Two-point crossover: Swap a continuous block between two points.
    Allows larger structural changes in schedules.
    """
    p1 = random.randint(1, len(parent1) - 3)
    p2 = random.randint(p1 + 1, len(parent1) - 2)

    child1 = parent1[:p1] + parent2[p1:p2] + parent1[p2:]
    child2 = parent2[:p1] + parent1[p1:p2] + parent2[p2:]

    return child1, child2


def order_crossover(parent1, parent2):
    """
    Order crossover (OX): Preserves relative order of genes.
    Useful for permutation-based representations.
    """
    size = len(parent1)
    p1 = random.randint(0, size - 2)
    p2 = random.randint(p1 + 1, size - 1)

    child = [None] * size
    child[p1:p2] = parent1[p1:p2]

    p2_genes = [gene for gene in parent2 if gene not in child]

    idx = 0
    for i in range(size):
        if child[i] is None:
            child[i] = p2_genes[idx]
            idx += 1

    return child


## Cell 8: Mutation Methods


In [8]:
def swap_mutation(individual):
    """
    Swap mutation: Exchange two randomly selected genes.
    Introduces small local changes while preserving feasibility.
    """
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]


def inversion_mutation(individual):
    """
    Inversion mutation: Reverse the order of a segment.
    Helps escape local optima by restructuring match order.
    """
    i, j = sorted(random.sample(range(len(individual)), 2))
    individual[i:j] = reversed(individual[i:j])


def scramble_mutation(individual):
    """
    Scramble mutation: Randomly shuffle a segment.
    Introduces more randomness than swap or inversion.
    """
    i, j = sorted(random.sample(range(len(individual)), 2))
    subset = individual[i:j]
    random.shuffle(subset)
    individual[i:j] = subset


def apply_mutation(individual, mutation_rate=0.1, method="swap"):
    """
    Apply mutation to individual based on mutation rate and method.
    """
    if random.random() > mutation_rate:
        return

    if method == "swap":
        swap_mutation(individual)
    elif method == "inversion":
        inversion_mutation(individual)
    elif method == "scramble":
        scramble_mutation(individual)


## Cell 9: Genetic Algorithm Function


In [9]:
def genetic_algorithm(teams, venues, dates, match_times,
                      pop_size=50, generations=100,
                      crossover_rate=0.8, mutation_rate=0.1,
                      elitism_count=2, tournament_size=3,
                      crossover_method="two_point",
                      mutation_method="swap",
                      selection_method="tournament",
                      callback=None):
    """
    Main Genetic Algorithm function.
    
    Parameters:
    -----------
    teams : list - List of team names
    venues : list - List of venue names
    dates : list - List of available dates
    match_times : list - List of match times (e.g., ["17:00", "20:00"])
    pop_size : int - Population size
    generations : int - Number of generations
    crossover_rate : float - Probability of crossover (0.0-1.0)
    mutation_rate : float - Probability of mutation (0.0-1.0)
    elitism_count : int - Number of elite individuals to preserve
    tournament_size : int - Size of tournament for selection
    crossover_method : str - "one_point", "two_point", or "order"
    mutation_method : str - "swap", "inversion", or "scramble"
    selection_method : str - "tournament" or "roulette"
    callback : function - Optional callback for progress updates
    
    Returns:
    --------
    dict with keys:
        - best_individual: Best schedule found
        - best_fitness: Fitness of best schedule
        - best_fitness_history: List of best fitness per generation
        - avg_fitness_history: List of average fitness per generation
        - worst_fitness_history: List of worst fitness per generation
    """
    print(f"\n{'='*60}")
    print("🧬 GENETIC ALGORITHM STARTED")
    print(f"{'='*60}")

    # Create initial population
    population = create_initial_population(pop_size, teams, venues, dates, match_times)

    # History tracking
    best_fitness_history = []
    avg_fitness_history = []
    worst_fitness_history = []
    best_individual = None
    best_fitness = 0

    # Evolution loop
    for gen in range(generations):
        # Calculate fitness for all individuals
        fitness_scores = [fitness(ind) for ind in population]

        gen_best = max(fitness_scores)
        gen_avg = sum(fitness_scores) / len(fitness_scores)
        gen_worst = min(fitness_scores)

        best_fitness_history.append(gen_best)
        avg_fitness_history.append(gen_avg)
        worst_fitness_history.append(gen_worst)

        # Track best individual
        if gen_best > best_fitness:
            best_fitness = gen_best
            best_individual = copy.deepcopy(
                population[fitness_scores.index(gen_best)]
            )

        # Progress output
        if gen % 10 == 0 or gen == generations - 1:
            print(f"Gen {gen:3d} | Best: {gen_best:.6f} | Avg: {gen_avg:.6f}")
            
        # Callback for GUI updates
        if callback:
            callback(gen, generations, gen_best, gen_avg)

        # ================= ELITISM =================
        elite_idx = sorted(
            range(len(fitness_scores)),
            key=lambda i: fitness_scores[i],
            reverse=True
        )[:elitism_count]

        new_population = [copy.deepcopy(population[i]) for i in elite_idx]

        # ================= CREATE OFFSPRING =================
        while len(new_population) < pop_size:

            # -------- Parent Selection --------
            if selection_method == "tournament":
                p1 = tournament_selection(population, fitness_scores, tournament_size)
                p2 = tournament_selection(population, fitness_scores, tournament_size)
            else:  # roulette
                p1 = roulette_wheel_selection(population, fitness_scores)
                p2 = roulette_wheel_selection(population, fitness_scores)

            # -------- Crossover --------
            if random.random() < crossover_rate:
                if crossover_method == "one_point":
                    c1, c2 = one_point_crossover(copy.deepcopy(p1), copy.deepcopy(p2))
                elif crossover_method == "two_point":
                    c1, c2 = two_point_crossover(copy.deepcopy(p1), copy.deepcopy(p2))
                else:  # order crossover
                    c1 = order_crossover(copy.deepcopy(p1), copy.deepcopy(p2))
                    c2 = order_crossover(copy.deepcopy(p2), copy.deepcopy(p1))
            else:
                c1, c2 = copy.deepcopy(p1), copy.deepcopy(p2)

            # -------- Mutation --------
            apply_mutation(c1, mutation_rate, mutation_method)
            apply_mutation(c2, mutation_rate, mutation_method)

            new_population.append(c1)
            if len(new_population) < pop_size:
                new_population.append(c2)

        population = new_population

    print(f"\n{'='*60}")
    print("✅ EVOLUTION COMPLETED")
    print(f"🏆 Best Fitness: {best_fitness:.6f}")
    print(f"{'='*60}\n")

    return {
        'best_individual': best_individual,
        'best_fitness': best_fitness,
        'best_fitness_history': best_fitness_history,
        'avg_fitness_history': avg_fitness_history,
        'worst_fitness_history': worst_fitness_history
    }


In [10]:
def plot_fitness_evolution(results, title="GA Fitness Evolution"):
    """
    Plot fitness evolution over generations
    Shows Best, Average, and Worst fitness
    """
    plt.figure(figsize=(14, 7))
    
    generations = range(len(results['best_fitness_history']))
    
    # Plot lines
    plt.plot(generations, results['best_fitness_history'], 
             'g-', linewidth=2.5, label='Best Fitness', marker='o', markersize=3)
    plt.plot(generations, results['avg_fitness_history'], 
             'b--', linewidth=2, label='Average Fitness', marker='s', markersize=2)
    plt.plot(generations, results['worst_fitness_history'], 
             'r:', linewidth=1.5, label='Worst Fitness', marker='^', markersize=2)
    
    # Fill area between best and worst
    plt.fill_between(generations, 
                     results['worst_fitness_history'],
                     results['best_fitness_history'],
                     alpha=0.2, color='blue', label='Fitness Range')
    
    # Styling
    plt.xlabel('Generation', fontsize=13, fontweight='bold')
    plt.ylabel('Fitness Value', fontsize=13, fontweight='bold')
    plt.title(title, fontsize=15, fontweight='bold')
    plt.legend(loc='best', fontsize=11)
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Add improvement annotation
    improvement = ((results['best_fitness_history'][-1] - results['best_fitness_history'][0]) 
                   / results['best_fitness_history'][0] * 100)
    plt.text(0.5, 0.95, f'Improvement: {improvement:.1f}%', 
             transform=plt.gca().transAxes,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
             fontsize=11, verticalalignment='top', horizontalalignment='center')
    
    plt.tight_layout()
    plt.show()


def plot_penalty_evolution(results, title="Penalty Reduction Over Time"):
    """
    Plot penalty reduction (inverse of fitness)
    """
    plt.figure(figsize=(14, 7))
    
    generations = range(len(results['best_fitness_history']))
    
    # Convert fitness to penalty
    best_penalty = [1/f - 1 for f in results['best_fitness_history']]
    avg_penalty = [1/f - 1 for f in results['avg_fitness_history']]
    worst_penalty = [1/f - 1 for f in results['worst_fitness_history']]
    
    plt.plot(generations, best_penalty, 
             'g-', linewidth=2.5, label='Best (Lowest Penalty)', marker='o', markersize=3)
    plt.plot(generations, avg_penalty, 
             'b--', linewidth=2, label='Average Penalty', marker='s', markersize=2)
    plt.plot(generations, worst_penalty, 
             'r:', linewidth=1.5, label='Worst (Highest Penalty)', marker='^', markersize=2)
    
    plt.xlabel('Generation', fontsize=13, fontweight='bold')
    plt.ylabel('Total Penalty', fontsize=13, fontweight='bold')
    plt.title(title, fontsize=15, fontweight='bold')
    plt.legend(loc='best', fontsize=11)
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Add reduction annotation
    reduction = ((worst_penalty[0] - best_penalty[-1]) / worst_penalty[0] * 100)
    plt.text(0.5, 0.95, f'Penalty Reduction: {reduction:.1f}%', 
             transform=plt.gca().transAxes,
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5),
             fontsize=11, verticalalignment='top', horizontalalignment='center')
    
    plt.tight_layout()
    plt.show()


def analyze_constraints(individual, teams):
    """
    Detailed constraint analysis
    Returns breakdown of penalties
    """
    print("\n" + "="*70)
    print("📊 DETAILED CONSTRAINT ANALYSIS")
    print("="*70)
    
    # 1. Venue conflicts
    venue_time_dict = {}
    venue_conflicts = 0
    conflict_details = []
    
    for match in individual:
        key = (match.date, match.time, match.venue)
        if key in venue_time_dict:
            venue_conflicts += 1
            conflict_details.append((match, venue_time_dict[key]))
        else:
            venue_time_dict[key] = match
    
    print(f"\n🏟️  VENUE CONFLICTS: {venue_conflicts}")
    print(f"   Penalty: {venue_conflicts * 5}")
    if venue_conflicts > 0 and venue_conflicts <= 3:
        print("   Examples:")
        for i, (m1, m2) in enumerate(conflict_details[:3], 1):
            print(f"   {i}. {m1.venue} on {m1.date.strftime('%Y-%m-%d')} at {m1.time}")
            print(f"      - {m1.team1} vs {m1.team2}")
            print(f"      - {m2.team1} vs {m2.team2}")
    
    # 2. Rest period violations
    team_dates = defaultdict(list)
    for match in individual:
        team_dates[match.team1].append(match.date)
        team_dates[match.team2].append(match.date)
    
    rest_violations = 0
    rest_violations_by_team = {}
    
    for team, dates_list in team_dates.items():
        dates_list.sort()
        team_violations = 0
        for i in range(1, len(dates_list)):
            days_rest = (dates_list[i] - dates_list[i-1]).days
            if days_rest < 1:
                rest_violations += 1
                team_violations += 1
        if team_violations > 0:
            rest_violations_by_team[team] = team_violations
    
    print(f"\n😴 REST PERIOD VIOLATIONS: {rest_violations}")
    print(f"   Penalty: {rest_violations * 3}")
    print(f"   Teams affected: {len(rest_violations_by_team)}/{len(teams)}")
    if rest_violations_by_team:
        worst_teams = sorted(rest_violations_by_team.items(), 
                           key=lambda x: x[1], reverse=True)[:5]
        print("   Worst 5 teams:")
        for team, count in worst_teams:
            print(f"   - {team}: {count} violations")
    
    # 3. Time balance
    team_time_count = {team: {"17:00": 0, "20:00": 0} for team in teams}
    for match in individual:
        team_time_count[match.team1][match.time] += 1
        team_time_count[match.team2][match.time] += 1
    
    total_imbalance = 0
    imbalanced_teams = {}
    
    for team, counts in team_time_count.items():
        imbalance = abs(counts["17:00"] - counts["20:00"])
        total_imbalance += imbalance
        if imbalance > 0:
            imbalanced_teams[team] = (counts["17:00"], counts["20:00"], imbalance)
    
    print(f"\n⏰ TIME IMBALANCE: {total_imbalance}")
    print(f"   Penalty: {total_imbalance * 1}")
    if imbalanced_teams:
        worst_imbalance = sorted(imbalanced_teams.items(), 
                                key=lambda x: x[1][2], reverse=True)[:5]
        print("   Worst 5 teams:")
        for team, (early, late, imb) in worst_imbalance:
            print(f"   - {team}: {early} early vs {late} late (diff: {imb})")
    
    # Total
    total_penalty = venue_conflicts * 5 + rest_violations * 3 + total_imbalance
    fitness_val = 1 / (1 + total_penalty)
    
    print(f"\n" + "="*70)
    print(f"💰 TOTAL PENALTY: {total_penalty}")
    print(f"🎯 FITNESS VALUE: {fitness_val:.6f}")
    print("="*70)
    
    return {
        'venue_conflicts': venue_conflicts,
        'rest_violations': rest_violations,
        'time_imbalance': total_imbalance,
        'total_penalty': total_penalty,
        'fitness': fitness_val
    }


def plot_constraint_breakdown(constraint_data):
    """
    Pie chart showing penalty breakdown
    """
    venue_penalty = constraint_data['venue_conflicts'] * 5
    rest_penalty = constraint_data['rest_violations'] * 3
    time_penalty = constraint_data['time_imbalance'] * 1
    
    labels = ['Venue Conflicts', 'Rest Violations', 'Time Imbalance']
    sizes = [venue_penalty, rest_penalty, time_penalty]
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1']
    explode = (0.1, 0.05, 0)
    
    plt.figure(figsize=(10, 7))
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', shadow=True, startangle=90)
    plt.title('Penalty Distribution by Constraint Type', 
             fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()


In [11]:
class TournamentSchedulerGUI:
    def __init__(self, root, ga_function, teams, venues, dates, match_times):
        self.root = root
        self.ga_function = ga_function
        self.teams = teams
        self.venues = venues
        self.dates = dates
        self.match_times = match_times
        self.results = None
        
        self.root.title("🏆 Sports Tournament Scheduler - Genetic Algorithm")
        self.root.geometry("900x700")
        self.root.configure(bg='#f0f0f0')
        
        self.setup_gui()
    
    def setup_gui(self):
        """Setup GUI components"""
        
        # Title
        title_frame = tk.Frame(self.root, bg='#2c3e50', height=80)
        title_frame.pack(fill='x')
        title_frame.pack_propagate(False)
        
        title_label = tk.Label(
            title_frame,
            text="🏆 SPORTS TOURNAMENT SCHEDULER",
            font=('Arial', 20, 'bold'),
            bg='#2c3e50',
            fg='white'
        )
        title_label.pack(expand=True)
        
        subtitle_label = tk.Label(
            title_frame,
            text="Using Genetic Algorithm Optimization",
            font=('Arial', 12),
            bg='#2c3e50',
            fg='#ecf0f1'
        )
        subtitle_label.pack()
        
        # Main container
        main_container = tk.Frame(self.root, bg='#f0f0f0')
        main_container.pack(fill='both', expand=True, padx=20, pady=20)
        
        # Left panel - Parameters
        left_panel = tk.LabelFrame(
            main_container,
            text="⚙️ GA Parameters",
            font=('Arial', 12, 'bold'),
            bg='white',
            padx=15,
            pady=15
        )
        left_panel.grid(row=0, column=0, sticky='nsew', padx=(0, 10))
        
        # Right panel - Output
        right_panel = tk.LabelFrame(
            main_container,
            text="📊 Output & Results",
            font=('Arial', 12, 'bold'),
            bg='white',
            padx=15,
            pady=15
        )
        right_panel.grid(row=0, column=1, sticky='nsew')
        
        main_container.columnconfigure(0, weight=1)
        main_container.columnconfigure(1, weight=2)
        main_container.rowconfigure(0, weight=1)
        
        # ===== LEFT PANEL - Parameters =====
        row = 0
        
        # Population Size
        tk.Label(left_panel, text="Population Size:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.pop_size_var = tk.IntVar(value=30)
        pop_size_spin = tk.Spinbox(left_panel, from_=10, to=200, 
                                   textvariable=self.pop_size_var, width=15)
        pop_size_spin.grid(row=row, column=1, pady=5)
        row += 1
        
        # Generations
        tk.Label(left_panel, text="Generations:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.gen_var = tk.IntVar(value=50)
        gen_spin = tk.Spinbox(left_panel, from_=10, to=500, 
                             textvariable=self.gen_var, width=15)
        gen_spin.grid(row=row, column=1, pady=5)
        row += 1
        
        # Crossover Rate
        tk.Label(left_panel, text="Crossover Rate:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.crossover_rate_var = tk.DoubleVar(value=0.8)
        crossover_spin = tk.Spinbox(left_panel, from_=0.0, to=1.0, increment=0.1,
                                   textvariable=self.crossover_rate_var, width=15)
        crossover_spin.grid(row=row, column=1, pady=5)
        row += 1
        
        # Mutation Rate
        tk.Label(left_panel, text="Mutation Rate:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.mutation_rate_var = tk.DoubleVar(value=0.1)
        mutation_spin = tk.Spinbox(left_panel, from_=0.0, to=1.0, increment=0.01,
                                  textvariable=self.mutation_rate_var, width=15)
        mutation_spin.grid(row=row, column=1, pady=5)
        row += 1
        
        # Elitism Count
        tk.Label(left_panel, text="Elitism Count:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.elitism_var = tk.IntVar(value=2)
        elitism_spin = tk.Spinbox(left_panel, from_=0, to=20, 
                                 textvariable=self.elitism_var, width=15)
        elitism_spin.grid(row=row, column=1, pady=5)
        row += 1
        
        # Tournament Size
        tk.Label(left_panel, text="Tournament Size:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.tournament_var = tk.IntVar(value=3)
        tournament_spin = tk.Spinbox(left_panel, from_=2, to=10, 
                                    textvariable=self.tournament_var, width=15)
        tournament_spin.grid(row=row, column=1, pady=5)
        row += 1
        
        # Crossover Method
        tk.Label(left_panel, text="Crossover Method:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.crossover_method_var = tk.StringVar(value="two_point")
        crossover_combo = ttk.Combobox(left_panel, textvariable=self.crossover_method_var,
                                      values=["one_point", "two_point", "order"],
                                      state='readonly', width=13)
        crossover_combo.grid(row=row, column=1, pady=5)
        row += 1
        
        # Mutation Method
        tk.Label(left_panel, text="Mutation Method:", 
                font=('Arial', 10), bg='white').grid(row=row, column=0, sticky='w', pady=5)
        self.mutation_method_var = tk.StringVar(value="swap")
        mutation_combo = ttk.Combobox(left_panel, textvariable=self.mutation_method_var,
                                     values=["swap", "inversion", "scramble"],
                                     state='readonly', width=13)
        mutation_combo.grid(row=row, column=1, pady=5)
        row += 1
        
        # Separator
        ttk.Separator(left_panel, orient='horizontal').grid(
            row=row, column=0, columnspan=2, sticky='ew', pady=15
        )
        row += 1
        
        # Info labels
        info_frame = tk.Frame(left_panel, bg='white')
        info_frame.grid(row=row, column=0, columnspan=2, pady=10)
        
        tk.Label(info_frame, text=f"📌 Teams: {len(self.teams)}", 
                font=('Arial', 9), bg='white').pack(anchor='w')
        tk.Label(info_frame, text=f"📌 Venues: {len(self.venues)}", 
                font=('Arial', 9), bg='white').pack(anchor='w')
        total_matches = len(self.teams) * (len(self.teams) - 1)
        tk.Label(info_frame, text=f"📌 Total Matches: {total_matches}", 
                font=('Arial', 9), bg='white').pack(anchor='w')
        row += 1
        
        # Buttons
        button_frame = tk.Frame(left_panel, bg='white')
        button_frame.grid(row=row, column=0, columnspan=2, pady=20)
        
        self.run_button = tk.Button(
            button_frame,
            text="▶️  RUN GA",
            font=('Arial', 12, 'bold'),
            bg='#27ae60',
            fg='white',
            width=15,
            height=2,
            command=self.run_ga,
            cursor='hand2'
        )
        self.run_button.pack(pady=5)
        
        self.stop_button = tk.Button(
            button_frame,
            text="⏹  STOP",
            font=('Arial', 10),
            bg='#e74c3c',
            fg='white',
            width=15,
            command=self.stop_ga,
            cursor='hand2',
            state='disabled'
        )
        self.stop_button.pack(pady=5)
        
        # ===== RIGHT PANEL - Output =====
        # Output text area
        self.output_text = scrolledtext.ScrolledText(
            right_panel,
            wrap=tk.WORD,
            width=60,
            height=25,
            font=('Courier', 9),
            bg='#ecf0f1'
        )
        self.output_text.pack(fill='both', expand=True, pady=5)
        
        # Progress bar
        self.progress = ttk.Progressbar(
            right_panel,
            mode='indeterminate',
            length=400
        )
        self.progress.pack(pady=10)
        
        # Status label
        self.status_label = tk.Label(
            right_panel,
            text="Ready to run optimization...",
            font=('Arial', 10),
            bg='white',
            fg='#7f8c8d'
        )
        self.status_label.pack()
        
        # Action buttons
        action_frame = tk.Frame(right_panel, bg='white')
        action_frame.pack(pady=10)
        
        tk.Button(
            action_frame,
            text="📊 View Plots",
            font=('Arial', 9),
            command=self.view_plots,
            width=15
        ).pack(side='left', padx=5)
        
        tk.Button(
            action_frame,
            text="📅 View Schedule",
            font=('Arial', 9),
            command=self.view_schedule,
            width=15
        ).pack(side='left', padx=5)
        
        tk.Button(
            action_frame,
            text="📋 Save Results",
            font=('Arial', 9),
            command=self.save_results,
            width=15
        ).pack(side='left', padx=5)
    
    def log(self, message):
        """Add message to output text"""
        self.output_text.insert(tk.END, message + '\n')
        self.output_text.see(tk.END)
        self.output_text.update()
    
    def run_ga(self):
        """Run GA in separate thread"""
        self.run_button.config(state='disabled')
        self.stop_button.config(state='normal')
        self.output_text.delete(1.0, tk.END)
        self.progress.start()
        self.status_label.config(text="🔄 Running optimization...", fg='#f39c12')
        
        # Run in thread
        thread = threading.Thread(target=self.run_ga_thread)
        thread.daemon = True
        thread.start()
    
    def run_ga_thread(self):
        """Actually run GA"""
        try:
            self.log("="*60)
            self.log("🧬 Starting Genetic Algorithm...")
            self.log("="*60)
            
            # Get parameters
            params = {
                'pop_size': self.pop_size_var.get(),
                'generations': self.gen_var.get(),
                'crossover_rate': self.crossover_rate_var.get(),
                'mutation_rate': self.mutation_rate_var.get(),
                'elitism_count': self.elitism_var.get(),
                'tournament_size': self.tournament_var.get(),
                'crossover_method': self.crossover_method_var.get(),
                'mutation_method': self.mutation_method_var.get()
            }
            
            self.log(f"\n📋 Parameters:")
            for key, value in params.items():
                self.log(f"   {key}: {value}")
            
            self.log("\n" + "="*60)
            self.log("Evolution starting...\n")
            
            # Run GA
            self.results = self.ga_function(
                self.teams, self.venues, self.dates, self.match_times,
                **params
            )
            
            self.log("\n" + "="*60)
            self.log("✅ OPTIMIZATION COMPLETED!")
            self.log("="*60)
            self.log(f"\n🏆 Best Fitness: {self.results['best_fitness']:.6f}")
            self.log(f"💰 Final Penalty: {1/self.results['best_fitness'] - 1:.2f}")
            
            improvement = ((self.results['best_fitness'] - self.results['best_fitness_history'][0]) 
                          / self.results['best_fitness_history'][0] * 100)
            self.log(f"📈 Improvement: {improvement:.2f}%")
            
            self.progress.stop()
            self.status_label.config(text="✅ Optimization completed successfully!", fg='#27ae60')
            messagebox.showinfo("Success", "GA completed successfully!\nYou can now view results.")
            
        except Exception as e:
            self.log(f"\n❌ Error: {str(e)}")
            self.progress.stop()
            self.status_label.config(text="❌ Error occurred", fg='#e74c3c')
            messagebox.showerror("Error", f"An error occurred:\n{str(e)}")
        
        finally:
            self.run_button.config(state='normal')
            self.stop_button.config(state='disabled')
    
    def stop_ga(self):
        """Stop GA (placeholder)"""
        messagebox.showinfo("Stop", "GA will stop after current generation")
        self.stop_button.config(state='disabled')
    
    def view_plots(self):
        """
        Display GA performance plots inside GUI
        """
        if self.results is None:
            messagebox.showwarning("No Results", "Please run the Genetic Algorithm first.")
            return

        plot_window = tk.Toplevel(self.root)
        plot_window.title("GA Performance Plots")
        plot_window.geometry("800x600")

        fig, ax = plt.subplots(figsize=(8, 4))

        ax.plot(self.results['best_fitness_history'], label='Best Fitness', linewidth=2)
        ax.plot(self.results['avg_fitness_history'], label='Average Fitness', linewidth=2)
        ax.plot(self.results['worst_fitness_history'], label='Worst Fitness', linewidth=2)

        ax.set_xlabel("Generation")
        ax.set_ylabel("Fitness")
        ax.set_title("Genetic Algorithm Performance")
        ax.legend()
        ax.grid(True)

        canvas = FigureCanvasTkAgg(fig, master=plot_window)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def view_schedule(self):
        """View schedule"""
        if self.results is None:
            messagebox.showwarning("Warning", "Please run GA first!")
            return
        
        # Create new window to show schedule
        schedule_window = tk.Toplevel(self.root)
        schedule_window.title("Tournament Schedule")
        schedule_window.geometry("900x600")
        
        # Scrolled text for schedule
        schedule_text = scrolledtext.ScrolledText(
            schedule_window,
            wrap=tk.WORD,
            font=('Courier', 9)
        )
        schedule_text.pack(fill='both', expand=True, padx=10, pady=10)
        
        # Print schedule
        best = self.results['best_individual']
        schedule_text.insert(tk.END, "="*90 + "\n")
        schedule_text.insert(tk.END, "📅 TOURNAMENT SCHEDULE\n")
        schedule_text.insert(tk.END, "="*90 + "\n\n")
        
        for i, match in enumerate(best[:50], 1):  # First 50 matches
            schedule_text.insert(tk.END, 
                f"{i}. {match.team1} vs {match.team2} (Leg {match.leg})\n"
                f"   {match.date.strftime('%Y-%m-%d')} at {match.time} - {match.venue}\n\n"
            )
        
        if len(best) > 50:
            schedule_text.insert(tk.END, f"\n... and {len(best) - 50} more matches\n")
    
    def save_results(self):
        """Save results to file"""
        if self.results is None:
            messagebox.showwarning("Warning", "Please run GA first!")
            return
        
        try:
            import csv
            with open('ga_results.csv', 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['Match#', 'Team1', 'Team2', 'Leg', 'Date', 'Time', 'Venue'])
                
                best = self.results['best_individual']
                for i, match in enumerate(best, 1):
                    writer.writerow([
                        i, match.team1, match.team2, match.leg,
                        match.date.strftime('%Y-%m-%d'), match.time, match.venue
                    ])
            
            messagebox.showinfo("Success", "Results saved to 'ga_results.csv'")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save: {str(e)}")


def launch_gui(ga_function, teams, venues, dates, match_times):
    """Launch the GUI"""
    root = tk.Tk()
    app = TournamentSchedulerGUI(root, ga_function, teams, venues, dates, match_times)
    root.mainloop()


In [ ]:
# Launch the GUI
launch_gui(genetic_algorithm, TEAMS, VENUES, DATES, MATCH_TIMES)



🧬 GENETIC ALGORITHM STARTED
Gen   0 | Best: 0.005376 | Avg: 0.004413
Gen  10 | Best: 0.007143 | Avg: 0.007143
Gen  20 | Best: 0.007143 | Avg: 0.007027
Gen  30 | Best: 0.007143 | Avg: 0.007071
Gen  40 | Best: 0.007143 | Avg: 0.007105
Gen  49 | Best: 0.007143 | Avg: 0.006890

✅ EVOLUTION COMPLETED
🏆 Best Fitness: 0.007143


🧬 GENETIC ALGORITHM STARTED
Gen   0 | Best: 0.005525 | Avg: 0.004320
Gen  10 | Best: 0.007092 | Avg: 0.007084
Gen  20 | Best: 0.007092 | Avg: 0.007092
Gen  30 | Best: 0.007092 | Avg: 0.007092
Gen  40 | Best: 0.007092 | Avg: 0.007092
Gen  49 | Best: 0.007092 | Avg: 0.007092

✅ EVOLUTION COMPLETED
🏆 Best Fitness: 0.007092


🧬 GENETIC ALGORITHM STARTED
Gen   0 | Best: 0.005464 | Avg: 0.004419
Gen  10 | Best: 0.008065 | Avg: 0.007906
Gen  20 | Best: 0.008264 | Avg: 0.008264
Gen  30 | Best: 0.008264 | Avg: 0.008264
